In [0]:
train=read.csv("/tmp/Train.csv",header = TRUE)
names(train)

In [0]:
train= train[c(1,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21)]

In [0]:
names(train)

In [0]:
ntr = train[c(3:17)]
names(ntr)

In [0]:
casdata=ntr[c(1,2,3,4,5,6,7,8,9,10,11,12,13)]
regdata=ntr[c(1,2,3,4,5,6,7,8,9,10,11,12,14)]
names(casdata)
names(regdata)

In [0]:
#install.packages("caret")
install.packages("Metrics")

In [0]:
#update.packages(ask = FALSE)
#detach("rlang")
#require(rlang)
#remove.packages("rlang")

In [0]:

#library(caTools) 
#library(caret)
#library(Metrics)
#library(rpart)
library(randomForest)


In [0]:
casdata$casual=casdata$casual+1
casdata$casual=log(casdata$casual)
regdata$registered=regdata$registered+1
regdata$registered=log(regdata$registered)

In [0]:
sample = sample.split(casdata,SplitRatio = 0.75) 
train_cas =subset(casdata,sample ==TRUE) 
test_cas=subset(casdata, sample==FALSE)

In [0]:
sample = sample.split(regdata,SplitRatio = 0.75) 
train_reg =subset(regdata,sample ==TRUE) 
test_reg=subset(regdata, sample==FALSE)

In [0]:
names(train_reg)

In [0]:
cas_linearMod <- lm(casual ~ ., data=train_cas)

cas_distPred <- predict(cas_linearMod, test_cas)

actual=test_cas$casual
pred=cas_distPred

compare=data.frame(cbind(sse=sse(actual,pred),mse=mse(actual,pred),rmse=rmse(actual,pred),rse=rse(actual,pred), mae=mae(actual,pred),rae=rae(actual,pred),accuracy=1-sum((actual - pred)^2)/sum((actual-mean(actual))^2)))
compare

In [0]:

reg_linearMod <- lm(registered ~ ., data=train_reg)

reg_distPred <- predict(reg_linearMod, test_reg)

actual=test_reg$registered
pred=reg_distPred

compare=data.frame(cbind(sse=sse(actual,pred),mse=mse(actual,pred),rmse=rmse(actual,pred),rse=rse(actual,pred), mae=mae(actual,pred),rae=rae(actual,pred),accuracy=1-sum((actual - pred)^2)/sum((actual-mean(actual))^2)))
compare

In [0]:
cas_DTR <- rpart(casual ~ ., data=train_cas,method = "anova") 

cas_distPred <- predict(cas_DTR, test_cas)

actual=test_cas$casual
pred=cas_distPred

compare=data.frame(cbind(sse=sse(actual,pred),mse=mse(actual,pred),rmse=rmse(actual,pred),rse=rse(actual,pred), mae=mae(actual,pred),rae=rae(actual,pred),accuracy=1-sum((actual - pred)^2)/sum((actual-mean(actual))^2)))
compare

In [0]:
reg_DTR <- rpart(train_reg$registered ~ ., data=train_reg,method = "anova")

reg_distPred <- predict(reg_DTR, test_reg)

actual=test_reg$registered
pred=reg_distPred

compare=data.frame(cbind(sse=sse(actual,pred),mse=mse(actual,pred),rmse=rmse(actual,pred),rse=rse(actual,pred), mae=mae(actual,pred),rae=rae(actual,pred),accuracy=1-sum((actual - pred)^2)/sum((actual-mean(actual))^2)))
compare

In [0]:
params_DTR_GS <- expand.grid(cp=c(0.1,0.001,0.01,0.02,0.03))
control = trainControl(method ="cv",number =3,search = "grid")
cas_DTR <- train(casual ~ ., data=train_cas,method="rpart",tuneGrid=params_DTR_GS,trControl=control)
cas_DTR$bestTune


cas_distPred <- predict(cas_DTR, test_cas)

actual=test_cas$casual
pred=cas_distPred

compare=data.frame(cbind(sse=sse(actual,pred),mse=mse(actual,pred),rmse=rmse(actual,pred),rse=rse(actual,pred), mae=mae(actual,pred),rae=rae(actual,pred),accuracy=1-sum((actual - pred)^2)/sum((actual-mean(actual))^2)))
compare

In [0]:
params_DTR_GS <- expand.grid(cp=c(0.1,0.001,0.01,0.02,0.03))
control = trainControl(method ="cv",number =3,search = "grid")
reg_DTR <- train(registered ~ ., data=train_reg,method="rpart",tuneGrid=params_DTR_GS,trControl=control)
reg_DTR$bestTune
reg_distPred <- predict(reg_DTR, test_reg)

actual=test_reg$registered
pred=reg_distPred

compare=data.frame(cbind(sse=sse(actual,pred),mse=mse(actual,pred),rmse=rmse(actual,pred),rse=rse(actual,pred), mae=mae(actual,pred),rae=rae(actual,pred),accuracy=1-sum((actual - pred)^2)/sum((actual-mean(actual))^2)))
compare

In [0]:
params_knn <- expand.grid(k=c(5:20))
control = trainControl(method ="cv",number =3,search = "grid")
cas_knn <- train(casual ~ ., data=train_cas,method="knn",tuneGrid=params_knn,trControl=control)

cas_knn$bestTune

cas_distPred <- predict(cas_knn, test_cas)

actual=test_cas$casual
pred=cas_distPred

compare=data.frame(cbind(sse=sse(actual,pred),mse=mse(actual,pred),rmse=rmse(actual,pred),rse=rse(actual,pred), mae=mae(actual,pred),rae=rae(actual,pred),accuracy=1-sum((actual - pred)^2)/sum((actual-mean(actual))^2)))
compare

In [0]:

params_knn <- expand.grid(k=c(5:20))
control = trainControl(method ="cv",number =3,search = "grid")
reg_knn <- train(registered ~ ., data=train_reg,method="knn",tuneGrid=params_knn,trControl=control)

reg_knn$bestTune
reg_distPred <- predict(reg_knn, test_reg)

actual=test_reg$registered
pred=reg_distPred

compare=data.frame(cbind(sse=sse(actual,pred),mse=mse(actual,pred),rmse=rmse(actual,pred),rse=rse(actual,pred), mae=mae(actual,pred),rae=rae(actual,pred),accuracy=1-sum((actual - pred)^2)/sum((actual-mean(actual))^2)))
compare

In [0]:
sqrt = sqrt(ncol(train_cas))
log2 = log2(ncol(train_cas))
n_features = 12
control = trainControl(method ="cv",number =10,search = "grid")
params_RFR = expand.grid(mtry = c(sqrt,log2, n_features))
cas_RF <- train(casual ~ ., data=train_cas,method="rf",tuneGrid=params_RFR,trControl=control)

cas_RF$bestTune

cas_distPred <- predict(cas_RF, test_cas)

actual=test_cas$casual
pred=cas_distPred

compare=data.frame(cbind(sse=sse(actual,pred),mse=mse(actual,pred),rmse=rmse(actual,pred),rse=rse(actual,pred), mae=mae(actual,pred),rae=rae(actual,pred),accuracy=1-sum((actual - pred)^2)/sum((actual-mean(actual))^2)))
compare

In [0]:
sqrt = sqrt(ncol(train_reg))
log2 = log2(ncol(train_reg))
n_features = 12
control = trainControl(method ="cv",number =10,search = "grid")
params_RFR = expand.grid(mtry = c(sqrt,log2, n_features))
reg_RF <- train(registered ~ ., data=train_reg,method="rf",tuneGrid=params_RFR,trControl=control)


reg_RF$bestTune
reg_distPred <- predict(reg_RF, test_reg)

actual=test_reg$registered
pred=reg_distPred

compare=data.frame(cbind(sse=sse(actual,pred),mse=mse(actual,pred),rmse=rmse(actual,pred),rse=rse(actual,pred), mae=mae(actual,pred),rae=rae(actual,pred),accuracy=1-sum((actual - pred)^2)/sum((actual-mean(actual))^2)))
compare